In [13]:
%pip install folium httpx rio-tiler titiler -q

Note: you may need to restart the kernel to use updated packages.


In [14]:
import json
import httpx
import pprint

from folium import Map, TileLayer

In [15]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.

url = "https://dea-public-data.s3-ap-southeast-2.amazonaws.com/projects/elevation/ga_srtm_dem1sv1_0/dem1sv1_0.tif" #GA DEM for Australia

## Get COG Info

In [16]:
# Fetch dataset Metadata
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

pprint.pprint(r)

{'band_descriptions': [['b1', 'dem1sv1_0']],
 'band_metadata': [['b1', {}]],
 'bounds': [112.99986111109999,
            -44.00013888924469,
            153.9998611117613,
            -10.000138888696277],
 'colorinterp': ['gray'],
 'count': 1,
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 122400,
 'maxzoom': 12,
 'minzoom': 2,
 'nodata_type': 'Nodata',
 'nodata_value': -3.4028234663852886e+38,
 'offsets': [0.0],
 'overviews': [2, 4, 8, 16, 32, 64, 128, 256],
 'scales': [1.0],
 'width': 147600}


### Display Tiles

By default, the tiles will be rescaled from min/max from dataset statistics (1615.812 / 2015.09448)

In [17]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=5
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Geoscience Australia Hydrologically enforced DEM 30m"
).add_to(m)
m

In [18]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        # rio-tiler cannot rescale automatically the data when using a colormap
        "rescale": "0, 2000",
        "colormap_name": "terrain",
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=5
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Office fédéral de topographie swisstopo"
)
aod_layer.add_to(m)
m

## Show Available Algorithms

In [19]:
# Fetch algorithms
print("Available algorithm")
algorithms = httpx.get(f"{titiler_endpoint}/algorithms").json()
print(list(algorithms))

# Metadata from `Hillshade` algorithm
meta = httpx.get(f"{titiler_endpoint}/algorithms/hillshade").json()
print("Inputs")
print(meta["inputs"])
print("Outputs")
print(meta["outputs"])
print("Parameters")
print(meta["parameters"])

Available algorithm
['hillshade', 'contours', 'normalizedIndex', 'terrarium', 'terrainrgb']
Inputs
{'nbands': 1}
Outputs
{'nbands': 1, 'dtype': 'uint8', 'min': None, 'max': None}
Parameters
{'azimuth': {'default': 90, 'maximum': 360, 'minimum': 0, 'title': 'Azimuth', 'type': 'integer'}, 'angle_altitude': {'default': 90.0, 'maximum': 90.0, 'minimum': -90.0, 'title': 'Angle Altitude', 'type': 'number'}, 'buffer': {'default': 3, 'maximum': 99, 'minimum': 0, 'title': 'Buffer', 'type': 'integer'}}


## Display Hillshade Tiles

In [20]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "algorithm": "hillshade",
        # Hillshade algorithm use a 3pixel buffer so we need
        # to tell the tiler to apply a 3 pixel buffer around each tile
        "buffer": 3,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=5
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
)
aod_layer.add_to(m)
m

### Pass parameters to the algorithm

In [27]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "algorithm": "contours",
        "algorithm_params": json.dumps(
            {
                "increment": 50, # contour line every 20 meters
                "thickness": 2, # 2m thickness
                "minz": 0,
                "maxz": 2000
            }
        ),
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
).add_to(m)
m